In [24]:
# Cell 1: Import necessary modules
import pandas as pd
import mysql.connector
from mysql.connector import Error
from getpass import getpass
import matplotlib.pyplot as plt

In [5]:
# Cell 2: Database configuration (fill these as needed)
host = 'localhost'
user = input('Enter your MySQL username: ')
password = getpass('Enter your MySQL password: ')
database = 'supermarket_db'


In [6]:
# Cell 3: Load your CSV into a DataFrame
df = pd.read_csv('supermarketsales.csv')

# Rename columns
df.columns = [
    'invoice_id', 'branch', 'city', 'customer_type', 'gender',
    'product_line', 'unit_price', 'quantity', 'tax_5_percent', 'total',
    'sale_date', 'sale_time', 'payment_method', 'cogs',
    'gross_margin_percentage', 'gross_income', 'rating'
]

# Convert date/time columns
df['sale_date'] = pd.to_datetime(df['sale_date']).dt.strftime('%Y-%m-%d')
df['sale_time'] = pd.to_datetime(df['sale_time']).dt.strftime('%H:%M:%S')

df.head()


/tmp/ipykernel_310365/2712869206.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sale_time'] = pd.to_datetime(df['sale_time']).dt.strftime('%H:%M:%S')


,invoice_id,branch,city,customer_type,gender,product_line,unit_price,quantity,tax_5_percent,total,sale_date,sale_time,payment_method,cogs,gross_margin_percentage,gross_income,rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,2019-01-05,13:08:00,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,2019-03-08,10:29:00,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,2019-03-03,13:23:00,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,2019-01-27,20:33:00,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2019-02-08,10:37:00,Ewallet,604.17,4.761905,30.2085,5.3


In [7]:
# Cell 4: Connect to MySQL and insert records
try:
    conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    if conn.is_connected():
        print('Connected to MySQL database')
        cursor = conn.cursor()
        
        insert_query = """
        INSERT INTO sales (
            invoice_id, branch, city, customer_type, gender, product_line,
            unit_price, quantity, tax_5_percent, total, sale_date, sale_time,
            payment_method, cogs, gross_margin_percentage, gross_income, rating
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        # Ensure all data types are native Python for MySQL
        data_to_insert = [tuple(row) for row in df.itertuples(index=False, name=None)]
        cursor.executemany(insert_query, data_to_insert)
        conn.commit()
        print(f"{cursor.rowcount} records inserted successfully into sales table.")

except Error as e:
    print(f"Error: {e}")
finally:
    if 'conn' in locals() and conn.is_connected():
        cursor.close()
        conn.close()
        print("MySQL connection is closed")


Connected to MySQL database
1000 records inserted successfully into sales table.
MySQL connection is closed


In [10]:
# Cell 2: Database configuration (fill these as needed)
host = 'localhost'
user = input('Enter your MySQL username: ')
password = getpass('Enter your MySQL password: ')
database = 'supermarket_db'

In [13]:
db_config = {
    'host': 'localhost',
    'user': 'root',  # e.g., 'root'
    'password': 'mysql',
    'database': 'supermarket_db'
}
# Connect to the database
mydb = mysql.connector.connect(**db_config)
mycursor = mydb.cursor()
print(mycursor)

MySQLCursor: (Nothing executed yet)


In [14]:
print("--- Supermarket Sales Analysis ---")

# Question 1: Show a few sample records
# Concept: SELECT * and fetchall()
print("\n[Question 1: Displaying first 5 sales records]")
mycursor.execute("SELECT * FROM sales LIMIT 5")
myresult = mycursor.fetchall()
for row in myresult:
    print(row)

--- Supermarket Sales Analysis ---

[Question 1: Displaying first 5 sales records]
('101-17-6199', 'A', 'Yangon', 'Normal', 'Male', 'Food and beverages', Decimal('45.79'), 7, Decimal('16.03'), Decimal('336.56'), datetime.date(2019, 3, 13), datetime.timedelta(seconds=71040), 'Credit card', Decimal('320.53'), 4.7619, Decimal('16.03'), 7.0)
('101-81-4070', 'C', 'Naypyitaw', 'Member', 'Female', 'Health and beauty', Decimal('62.82'), 2, Decimal('6.28'), Decimal('131.92'), datetime.date(2019, 1, 17), datetime.timedelta(seconds=45360), 'Ewallet', Decimal('125.64'), 4.7619, Decimal('6.28'), 4.9)
('102-06-2002', 'C', 'Naypyitaw', 'Member', 'Male', 'Sports and travel', Decimal('25.25'), 5, Decimal('6.31'), Decimal('132.56'), datetime.date(2019, 3, 20), datetime.timedelta(seconds=64320), 'Cash', Decimal('126.25'), 4.7619, Decimal('6.31'), 6.1)
('102-77-2261', 'C', 'Naypyitaw', 'Member', 'Male', 'Health and beauty', Decimal('65.31'), 7, Decimal('22.86'), Decimal('480.03'), datetime.date(2019, 3, 5

In [15]:
# Question 2: Which city has the most sales? 
# concept - SELECT with WHERE clause
# find all sales from a specific city
print("\n[Question 2: Finding all sales from the 'Yangon' branch]")
sql = "SELECT invoice_id, product_line, total FROM sales WHERE city = 'Yangon'"
mycursor.execute(sql)
myresult = mycursor.fetchall()
print(f"Found {len(myresult)} sales in Yangon. Showing first 5:")
for row in myresult[:5]:
    print(f"Invoice: {row[0]}, Product: {row[1]}, Total: {row[2]}")



[Question 2: Finding all sales from the 'Yangon' branch]
Found 340 sales in Yangon. Showing first 5:
Invoice: 101-17-6199, Product: Food and beverages, Total: 336.56
Invoice: 105-10-6182, Product: Fashion accessories, Total: 45.11
Invoice: 105-31-1824, Product: Sports and travel, Total: 510.97
Invoice: 106-35-6779, Product: Home and lifestyle, Total: 93.11
Invoice: 123-19-1176, Product: Health and beauty, Total: 489.05


In [16]:
# Question 2: Which city has the most sales? 
# concept - SELECT with WHERE clause
# find all sales from a specific city
print("\n[Question 2: Finding all sales from the 'Naypyitaw' branch]")
sql = "SELECT invoice_id, product_line, total FROM sales WHERE city = 'Naypyitaw'"
mycursor.execute(sql)
myresult = mycursor.fetchall()
print(f"Found {len(myresult)} sales in Naypyitaw. Showing first 5:")
for row in myresult[:5]:
    print(f"Invoice: {row[0]}, Product: {row[1]}, Total: {row[2]}")



[Question 2: Finding all sales from the 'Naypyitaw' branch]
Found 328 sales in Naypyitaw. Showing first 5:
Invoice: 101-81-4070, Product: Health and beauty, Total: 131.92
Invoice: 102-06-2002, Product: Sports and travel, Total: 132.56
Invoice: 102-77-2261, Product: Health and beauty, Total: 480.03
Invoice: 110-05-6330, Product: Food and beverages, Total: 248.41
Invoice: 115-38-7388, Product: Fashion accessories, Total: 85.51


In [17]:
# Question 3: What are the top 5 highest-value sales?
# concept : Order by
print("\n[Question 3: Top 5 highest-value transactions]")
sql = "SELECT invoice_id, city, total FROM sales ORDER BY total DESC LIMIT 5"
mycursor.execute(sql)
myresult = mycursor.fetchall()
for row in myresult:
    print(f"Invoice: {row[0]}, City: {row[1]}, Total Sale: {row[2]}")


[Question 3: Top 5 highest-value transactions]
Invoice: 860-79-0874, City: Naypyitaw, Total Sale: 1042.65
Invoice: 687-47-8271, City: Yangon, Total Sale: 1039.29
Invoice: 283-26-5248, City: Naypyitaw, Total Sale: 1034.46
Invoice: 751-41-9720, City: Naypyitaw, Total Sale: 1023.75
Invoice: 303-96-2227, City: Mandalay, Total Sale: 1022.49


In [18]:
# Question 4: Which sales were made to 'Normal' customers for 'Fashion accessories'? 
# Concept: Combining WHERE with AND
print("\n[Question 4: Sales to 'Normal' customers for 'Fashion accessories']")
sql = "SELECT invoice_id, total, rating FROM sales WHERE customer_type = 'Normal' AND product_line = 'Fashion accessories'"
mycursor.execute(sql)
myresult = mycursor.fetchall()
print(f"Found {len(myresult)} such sales. Showing first 5:")
for row in myresult[:5]:
    print(f"Invoice: {row[0]}, Total: {row[1]}, Rating: {row[2]}")


[Question 4: Sales to 'Normal' customers for 'Fashion accessories']
Found 92 such sales. Showing first 5:
Invoice: 125-45-2293, Total: 624.33, Rating: 4.2
Invoice: 130-98-8941, Total: 472.31, Rating: 5.7
Invoice: 135-84-8019, Total: 736.44, Rating: 7.6
Invoice: 137-74-8729, Total: 102.40, Rating: 6.8
Invoice: 139-52-2867, Total: 165.45, Rating: 4.8


In [19]:
# Question 5: Find all sales of products with 'health' in the name
# Concept: WHERE with LIKE (Wildcards)
print("\n[Question 5: Finding sales from product lines containing 'Health']")
sql = "SELECT invoice_id, product_line, total FROM sales WHERE product_line LIKE '%Health%'"
mycursor.execute(sql)
myresult = mycursor.fetchall()
for row in myresult:
    print(f"Invoice: {row[0]}, Product Line: {row[1]}, Total: {row[2]}")


[Question 5: Finding sales from product lines containing 'Health']
Invoice: 101-81-4070, Product Line: Health and beauty, Total: 131.92
Invoice: 102-77-2261, Product Line: Health and beauty, Total: 480.03
Invoice: 123-19-1176, Product Line: Health and beauty, Total: 489.05
Invoice: 127-47-6963, Product Line: Health and beauty, Total: 217.18
Invoice: 131-70-8179, Product Line: Health and beauty, Total: 290.08
Invoice: 132-23-6451, Product Line: Health and beauty, Total: 110.09
Invoice: 133-14-7229, Product Line: Health and beauty, Total: 132.03
Invoice: 148-41-7930, Product Line: Health and beauty, Total: 734.71
Invoice: 149-14-0304, Product Line: Health and beauty, Total: 239.40
Invoice: 152-08-9985, Product Line: Health and beauty, Total: 608.20
Invoice: 157-13-5295, Product Line: Health and beauty, Total: 545.37
Invoice: 160-22-2687, Product Line: Health and beauty, Total: 503.74
Invoice: 176-78-1170, Product Line: Health and beauty, Total: 106.50
Invoice: 186-09-3669, Product Line:

In [20]:
# Question 6: Show me a single high-rated transaction 
# Concept: fetchone()
print("\n[Question 6: Fetching a single high-rated transaction (rating > 9)]")
mycursor.execute("SELECT * FROM sales WHERE rating > 9.0 LIMIT 1")
single_result = mycursor.fetchone()
print("A single high-rated transaction:")
print(single_result)


[Question 6: Fetching a single high-rated transaction (rating > 9)]
A single high-rated transaction:
('109-28-2512', 'B', 'Mandalay', 'Member', 'Female', 'Fashion accessories', Decimal('97.61'), 6, Decimal('29.28'), Decimal('614.94'), datetime.date(2019, 1, 7), datetime.timedelta(seconds=54060), 'Ewallet', Decimal('585.66'), 4.7619, Decimal('29.28'), 9.9)


In [23]:
# 1. Sales by Branch/City
print("\n--- Q6: What are the total sales by branch? ---")
sql_branch = "SELECT branch, SUM(total) as total_sales FROM sales GROUP BY branch ORDER BY total_sales DESC"
mycursor.execute(sql_branch)
results = mycursor.fetchall()
for row in results:
    print(f"Branch {row[0]}: ${row[1]:.2f}")


--- Q6: What are the total sales by branch? ---
Branch C: $110568.86
Branch A: $106200.57
Branch B: $106198.00


In [25]:
sql_products = """
    SELECT product_line, SUM(total) as total_sales
    FROM sales
    GROUP BY product_line
    ORDER BY total_sales DESC
    LIMIT 5
"""
mycursor.execute(sql_products)
results = mycursor.fetchall()
print("\n--- Q2: Top 5 Selling Product Lines ---")
for product, sales in results:
    print(f"{product}: ${sales:.2f}")



--- Q2: Top 5 Selling Product Lines ---
Food and beverages: $56144.96
Sports and travel: $55123.00
Electronic accessories: $54337.64
Fashion accessories: $54306.03
Home and lifestyle: $53861.96


In [26]:
sql_customer = """
    SELECT customer_type,
           COUNT(*) as num_transactions,
           SUM(total) as total_sales,
           AVG(total) as avg_sale_value
    FROM sales
    GROUP BY customer_type
"""
mycursor.execute(sql_customer)
results = mycursor.fetchall()
print("\n--- Q3: Sales Difference Between 'Member' and 'Normal' Customers ---")
print(f"{'Customer Type':<15} {'Transactions':<15} {'Total Sales':<15} {'Avg. Sale'}")
print("-" * 60)
for ctype, num_trans, total_sales, avg_sale in results:
    print(f"{ctype:<15} {num_trans:<15} ${total_sales:<14.2f} ${avg_sale:.2f}")



--- Q3: Sales Difference Between 'Member' and 'Normal' Customers ---
Customer Type   Transactions    Total Sales     Avg. Sale
------------------------------------------------------------
Normal          499             $158743.62      $318.12
Member          501             $164223.81      $327.79


In [27]:
sql_payment = """
    SELECT payment_method, SUM(total) as total_sales
    FROM sales
    GROUP BY payment_method
    ORDER BY total_sales DESC
"""
mycursor.execute(sql_payment)
results = mycursor.fetchall()
print("\n--- Q4: Total Sales by Payment Method ---")
for method, sales in results:
    print(f"{method:<15}: ${sales:.2f}")



--- Q4: Total Sales by Payment Method ---
Cash           : $112206.76
Ewallet        : $109993.38
Credit card    : $100767.29


In [34]:
sql_rating = """
    SELECT gender, customer_type, AVG(rating) as avg_rating
    FROM sales
    GROUP BY gender, customer_type
    ORDER BY gender, customer_type
"""
mycursor.execute(sql_rating)
results = mycursor.fetchall()
print("\n--- Q8: Average Rating by Gender and Customer Type ---")
print(f"{'Gender':<10} {'Customer Type':<15} {'Average Rating'}")
print("-" * 45)
for gender, ctype, avg_rating in results:
    print(f"{gender:<10} {ctype:<15} {avg_rating:.2f}")


InternalError: Unread result found

In [ ]:
sql_margin = """
    SELECT product_line, AVG(gross_margin_percentage) as avg_margin
    FROM sales
    GROUP BY product_line
    ORDER BY avg_margin DESC
    LIMIT 5
"""
mycursor.execute(sql_margin)
results = mycursor.fetchall()
print("\n--- Q6: Product Lines With Highest Average Gross Margin ---")
for product, avg_margin in results:
    print(f"{product:<30}: {avg_margin:.2f}%")


InternalError: Unread result found

In [31]:
sql_trends = """
    SELECT sale_date, SUM(total) as daily_sales
    FROM sales
    GROUP BY sale_date
    ORDER BY sale_date ASC
"""


df_trends = pd.read_sql(sql_trends, mycursor.execute(sql_trends))

plt.figure(figsize=(12, 6))
plt.plot(df_trends['sale_date'], df_trends['daily_sales'], marker='o', linestyle='-')
plt.title('Daily Sales Trend Over Time')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

plot_filename = 'sales_trends.png'
plt.savefig(plot_filename)
print(f"Date trend plot saved as '{plot_filename}'")
# Uncomment to display in notebook:
# plt.show()


AttributeError: 'NoneType' object has no attribute 'cursor'